In [1]:
import numpy as np

In [10]:
def hillslopeHRU(hill, storages, parameters):
    # Are_Elevations gives the areal percentage of each elevation band. The sum has to be 1
    # Area_elevations, Precipitation, Temp_elevation, Snowstorage, Interceptionstorage has to be array of length Nr_Elevationbands
    assert hill.Total_Interception_Evaporation == 0
    assert hill.Total_Effective_Precipitation == 0
    assert 0.9999999 <= sum(hill.Area_Elevations) <= 1.000001 or sum(hill.Area_Elevations) == 0
    assert hill.Area_HRU >= 0 and hill.Area_HRU <= 1
    assert hill.Nr_Elevationbands >= 1
    
    # define Arrays for Snow and Interception storage
    Snow = np.zeros(hill.Nr_Elevationbands)
    Interception = np.zeros(hill.Nr_Elevationbands)
    Snow_Cover = np.zeros(hill.Nr_Elevationbands)
    Total_Melt_Glacier = 0
    
    # calculate the model components that are height dependend
    for i in range(hill.Nr_Elevationbands):
        # snow component
        Melt_Glacier, Melt, Snow[i] = snow(hill.Area_Glacier[i], hill.Precipitation[i], hill.Temp_Elevation[i], storages.Snow[i], parameters.Meltfactor, parameters.Mm, parameters.Temp_Thresh)
        #interception component
        Effective_Precipitation, Interception_Evaporation, Interception[i] = interception(hill.Potential_Evaporation_Mean, hill.Precipitation[i], hill.Temp_Elevation[i], storages.Interception[i], parameters.Interceptionstoragecapacity, parameters.Temp_Thresh)
        # the melt, effective precipitation and evaporation can be summed up over all elevations according to the areal extent
        hill.Total_Effective_Precipitation += (Effective_Precipitation + Melt) * hill.Area_Elevations[i]
        #global Total_Melt += (Melt * Area_Elevations[i])
        hill.Total_Interception_Evaporation += (Interception_Evaporation * hill.Area_Elevations[i])
        Total_Melt_Glacier += (Melt_Glacier * hill.Area_Elevations[i])
        # get the extend of snow cover, if there is snow in the snow bucket the snow cover is 1
        if Snow[i] > 1:
            Snow_Cover[i] = 1
        else:
            Snow_Cover[i] = hill.Area_Glacier[i]
            
    #soil storage component
    Overlandflow, Preferentialflow, Soil_Evaporation, Soilstorage = soilstorage(hill.Total_Effective_Precipitation, hill.Total_Interception_Evaporation, hill.Potential_Evaporation_Mean, storages.Soil, parameters.beta, parameters.Ce, parameters.Ratio_Pref, parameters.Soilstoragecapacity)
    GWflow = Preferentialflow
    #fast storage
    Fast_Discharge, Faststorage = faststorage(Overlandflow, storages.Fast, parameters.Kf)

    #calculate total outflow
    Flows = Fast_Discharge + GWflow + hill.Total_Interception_Evaporation + Soil_Evaporation
    assert hill.Potential_Evaporation_Mean - Soil_Evaporation - hill.Total_Interception_Evaporation >= - 5 * eps
    # change discharges and evaporation fluxed according to areal fraction
    Fast_Discharge = Fast_Discharge * hill.Area_HRU
    GWflow = GWflow * hill.Area_HRU
    Total_Interception_Evaporation = hill.Total_Interception_Evaporation * hill.Area_HRU
    Soil_Evaporation = Soil_Evaporation * hill.Area_HRU
    # returning all fluxes (evporative, discharge)
    hill_out = [Fast_Discharge, GWflow, Soil_Evaporation, Total_Interception_Evaporation] #Outflows(Fast_Discharge, GWflow, Soil_Evaporation, Total_Interception_Evaporation)
    #returning all storages
    hill_storages = [Faststorage, Interception, Snow, Snow_Cover, Soilstorage]#Storages(Faststorage, Interception, Snow, Snow_Cover, Soilstorage)
    # total interception should be zero again for next run
    hill.Total_Interception_Evaporation = 0
    hill.Total_Effective_Precipitation = 0

    #assertions for the outflows
    assert hill_out.Fast_Discharge >= 0
    assert hill_out.GWflow >= 0
    assert hill_out.Soil_Evaporation >= 0
    assert hill_out.Interception_Evaporation >= 0
    #assertions for the storages
    assert hill_storages.Fast >= 0
    assert hill_storages.Interception >= zeros(hill.Nr_Elevationbands)
    assert hill_storages.Snow >= zeros(hill.Nr_Elevationbands)
    assert hill_storages.Soil >= 0
    assert ones(hill.Nr_Elevationbands) * parameters.Interceptionstoragecapacity - hill_storages.Interception >= zeros(hill.Nr_Elevationbands)
    assert parameters.Soilstoragecapacity - hill_storages.Soil >= 0

    #assertion water balance
    Precipitation = 0
    Interception_Storage_New = 0
    Snow_Storage_New = 0
    Interception_Storage_Old = 0
    Snow_Storage_Old = 0
    # get total storage of interception and snow over all elevation zones
    for i in range(hill.Nr_Elevationbands):
        Precipitation += hill.Precipitation[i] * hill.Area_Elevations[i]
        Interception_Storage_New += hill_storages.Interception[i] * hill.Area_Elevations[i]
        Snow_Storage_New += hill_storages.Snow[i] * hill.Area_Elevations[i]
        Interception_Storage_Old += storages.Interception[i] * hill.Area_Elevations[i]
        Snow_Storage_Old += storages.Snow[i] * hill.Area_Elevations[i]
    
    Flows_Area = Fast_Discharge + GWflow + Total_Interception_Evaporation + Soil_Evaporation
    All_Storages = (hill_storages.Fast - storages.Fast) + (hill_storages.Soil - storages.Soil) + (Snow_Storage_New -Snow_Storage_Old) + (Interception_Storage_New - Interception_Storage_Old)

    #assert that water balance closes
    assert -0.00000001 <= Precipitation + Total_Melt_Glacier - (Flows + All_Storages) <= 0.00000001
    assert -0.00000001 <= (Precipitation + Total_Melt_Glacier)  * hill.Area_HRU - (Flows_Area + All_Storages * hill.Area_HRU) <= 0.00000001
    
    return hill_out, hill_storages, Precipitation, All_Storages

In [13]:
def riparianHRU(rip, storages, parameters):
    # Are_Elevations gives the areal percentage of each elevation band. The sum has to be 1
    # Area_elevations, Precipitation, Temp_elevation, Snowstorage, Interceptionstorage has to be array of length Nr_Elevationbands
    # riparian HRU has no preferential flow

    assert rip.Total_Interception_Evaporation == 0
    assert rip.Total_Effective_Precipitation == 0
    assert 0.9999999 <= sum(rip.Area_Elevations) <= 1.000001 or sum(rip.Area_Elevations) == 0
    assert rip.Area_HRU >= 0 and rip.Area_HRU <= 1
    assert rip.Nr_Elevationbands >= 1
    
    Snow = np.zeros(rip.Nr_Elevationbands)
    Interception = np.zeros(rip.Nr_Elevationbands)
    Snow_Cover = np.zeros(rip.Nr_Elevationbands)
    for i in range(rip.Nr_Elevationbands):
        # snow component
        assert rip.Area_Glacier[i] == 0.0
        Melt_Glacier, Melt, Snow[i] = snow(rip.Area_Glacier[i], rip.Precipitation[i], rip.Temp_Elevation[i], storages.Snow[i], parameters.Meltfactor, parameters.Mm, parameters.Temp_Thresh)
        assert Melt_Glacier == 0.0
        #interception component
        Effective_Precipitation, Interception_Evaporation, Interception[i] = interception(rip.Potential_Evaporation_Mean, rip.Precipitation[i], rip.Temp_Elevation[i], storages.Interception[i], parameters.Interceptionstoragecapacity, parameters.Temp_Thresh)
        # the melt, effective precipitation and evaporation can be summed up over all elevations according to the areal extent
        rip.Total_Effective_Precipitation += (Effective_Precipitation + Melt) * rip.Area_Elevations[i]
        #global Total_Melt += (Melt * Area_Elevations[i])
        rip.Total_Interception_Evaporation += Interception_Evaporation * rip.Area_Elevations[i]
        #get snow cover extent
        if Snow[i] > 1:
            Snow_Cover[i] = 1


    #soil storage component
    Overlandflow, Soil_Evaporation, Soilstorage = ripariansoilstorage(rip.Total_Effective_Precipitation, rip.Total_Interception_Evaporation, rip.Potential_Evaporation_Mean, rip.Riparian_Discharge / rip.Area_HRU, storages.Soil, parameters.beta, parameters.Ce, parameters.Drainagecapacity, parameters.Soilstoragecapacity)
    #fast storage
    Fast_Discharge, Faststorage = faststorage(Overlandflow, storages.Fast, parameters.Kf)
    GWflow = 0
    #calculate total outflow
    Flows = Fast_Discharge + GWflow + rip.Total_Interception_Evaporation + Soil_Evaporation
    assert rip.Potential_Evaporation_Mean - Soil_Evaporation - rip.Total_Interception_Evaporation >= - 5 * eps
    # change discharges according to areal fraction
    Fast_Discharge = Fast_Discharge * rip.Area_HRU
    Total_Interception_Evaporation = rip.Total_Interception_Evaporation * rip.Area_HRU
    Soil_Evaporation = Soil_Evaporation * rip.Area_HRU
    # return water flows, evaporation, and states of the storage components
    rip_out =  [Fast_Discharge, GWflow, Soil_Evaporation, Total_Interception_Evaporation]#Outflows(Fast_Discharge, GWflow, Soil_Evaporation, Total_Interception_Evaporation)
    rip_storages = [Faststorage, Interception, Snow, Snow_Cover, Soilstorage] #Storages(Faststorage, Interception, Snow, Snow_Cover, Soilstorage)

    rip.Total_Interception_Evaporation = 0
    rip.Total_Effective_Precipitation = 0

    #assertions for the outflows
    assert rip_out.Fast_Discharge >= 0
    assert rip_out.GWflow >= 0
    assert rip_out.Soil_Evaporation >= 0
    assert rip_out.Interception_Evaporation >= 0
    #assertions for the storages
    assert rip_storages.Fast >= 0
    assert rip_storages.Interception >= zeros(rip.Nr_Elevationbands)
    assert rip_storages.Snow >= zeros(rip.Nr_Elevationbands)
    assert rip_storages.Soil >= 0
    assert rip_storages.Interception <= ones(rip.Nr_Elevationbands) * parameters.Interceptionstoragecapacity
    #@assert ones(rip.Nr_Elevationbands) * parameters.Interceptionstoragecapacity - rip_storages.Interception >= -10^(-10) * ones(rip.Nr_Elevationbands)
    assert rip_storages.Soil <= parameters.Soilstoragecapacity
    #@assert parameters.Soilstoragecapacity - rip_storages.Soil >= -10^(-10)

    Precipitation = 0
    Interception_Storage_New = 0
    Snow_Storage_New = 0
    Interception_Storage_Old = 0
    Snow_Storage_Old = 0
    for i in range(rip.Nr_Elevationbands):
        Precipitation += rip.Precipitation[i] * rip.Area_Elevations[i]
        Interception_Storage_New += rip_storages.Interception[i] * rip.Area_Elevations[i]
        Snow_Storage_New += rip_storages.Snow[i] * rip.Area_Elevations[i]
        Interception_Storage_Old += storages.Interception[i] * rip.Area_Elevations[i]
        Snow_Storage_Old += storages.Snow[i] * rip.Area_Elevations[i]
   

    Flows_Area = Fast_Discharge + GWflow + Total_Interception_Evaporation + Soil_Evaporation
    All_Storages = (rip_storages.Fast - storages.Fast) + (rip_storages.Soil -storages.Soil) + (Snow_Storage_New -Snow_Storage_Old) + (Interception_Storage_New - Interception_Storage_Old)
    assert -0.00000001 <= Precipitation + rip.Riparian_Discharge / rip.Area_HRU - (Flows + All_Storages) <= 0.00000001
    assert -0.00000001 <= (Precipitation * rip.Area_HRU + rip.Riparian_Discharge) - (Flows_Area + All_Storages * rip.Area_HRU) <= 0.00000001

    return rip_out, rip_storages, Precipitation, All_Storages

